In [127]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
import os
from datasets import *
from utils import *
from wrapper import *
from typing import List, Dict

torch.cuda.is_available()

True

In [89]:
train_df = pd.read_csv('../data/train.csv', sep='\t')    # [700:900]
pos_df = train_df#[train_df.label == 1]

In [90]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-cluener2020-chinese")
model = AutoModelForTokenClassification.from_pretrained("uer/roberta-base-finetuned-cluener2020-chinese")
model.cuda()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
ner = pipeline('ner', model=model, tokenizer=tokenizer, device=device)
pos_text = pos_df.text

In [93]:
ner_outputs = ner(inputs=pos_text.values.tolist())
non_empty_idx = np.argwhere(ner_outputs).flatten()
ntf()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\numpy\core\fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [124]:
def postprocess_ds(outputs:List[List[dict]]):
    entity_vocab = {}
    for output in outputs:
        if output:
            sentence_vocab = postprocess_sentence(output)
            for k, v in sentence_vocab.items():
                if k in entity_vocab.keys():
                    for v_ in v:
                        if v_ not in entity_vocab[k]:
                            entity_vocab[k].append(v_) 
                else:
                    entity_vocab[k] = v
    return entity_vocab


def postprocess_sentence(ner_outputs:List[dict]):
    entity_vocab = {}
    if ner_outputs == []:
        return

    current = ''
    for out in ner_outputs:
        if out['entity'][0] == 'B':
            if len(current) > 1:
                # if category == 'scene': print(current)
                if category not in entity_vocab.keys():
                    entity_vocab[category] = []
                if current not in entity_vocab[category]:
                    entity_vocab[category].append(current)
            current = ''
            category = out['entity'][2:]
            current += out['word']
        if out['entity'][0] == 'I':
            if not current:
                continue
            current += out['word']
    if len(current) > 1:
        if category not in entity_vocab.keys():
            entity_vocab[category] = []
        if current not in entity_vocab[category]:
            entity_vocab[category].append(current)
    return entity_vocab

In [137]:
entity_vocab = postprocess_ds(ner_outputs)
output_file_name = os.path.join("D:\Apps\Anaconda3\envs\general-torch\Lib\site-packages", "nlpcda\data\entities.txt")

with open(output_file_name, 'w', encoding='utf-8') as f:
    for k, words in entity_vocab.items():
        f.write(' '.join(words) + '\n')


[[],
 [{'entity': 'B-organization',
   'score': 0.9305042,
   'index': 8,
   'word': '英',
   'start': 7,
   'end': 8},
  {'entity': 'I-organization',
   'score': 0.8825962,
   'index': 9,
   'word': '超',
   'start': 8,
   'end': 9},
  {'entity': 'B-organization',
   'score': 0.9830223,
   'index': 12,
   'word': '曼',
   'start': 11,
   'end': 12},
  {'entity': 'I-organization',
   'score': 0.9880143,
   'index': 13,
   'word': '城',
   'start': 12,
   'end': 13},
  {'entity': 'I-organization',
   'score': 0.93196356,
   'index': 14,
   'word': '队',
   'start': 13,
   'end': 14},
  {'entity': 'B-organization',
   'score': 0.93191606,
   'index': 24,
   'word': '英',
   'start': 29,
   'end': 30},
  {'entity': 'I-organization',
   'score': 0.9104762,
   'index': 25,
   'word': '超',
   'start': 30,
   'end': 31},
  {'entity': 'B-organization',
   'score': 0.8939709,
   'index': 31,
   'word': '英',
   'start': 36,
   'end': 37},
  {'entity': 'I-organization',
   'score': 0.8594923,
   'index